In [1044]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [1045]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [1046]:
ticker_list = ["aapl", "amzn", "msft"]

ticker = ticker_list[1]

ticker_upper = ticker.upper()

print(ticker_upper)

AMZN


In [1047]:
alphavantage_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}.csv"

In [1048]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek

df["adjusted_close_shift"] = df["5. adjusted close"].shift(-1)

df["percentage_change"] = (df["adjusted_close_shift"] - df["5. adjusted close"]) / df["5. adjusted close"] * 100.0

df["increase"] = df["percentage_change"].apply(lambda x: 1 if x > 0 else 0)

df.set_index("date", inplace=True)

In [1049]:
df.shape[0]

3064

In [1050]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  adjusted_close_shift  3063 non-null   float64       
 11  percentage_change     3063 non-null   float64       
 12  increase              3064 non-null   int64         
dtypes: datet

In [1051]:
df.tail()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase
date,,,,,,,,,,,,,
2022-02-28,3048.50,3089.00,3017.00,3071.26,3071.26,2884171.0,0.0,1.0,2022-02-28,0,3022.84,-1.576552,0
2022-03-01,3054.65,3081.98,2999.54,3022.84,3022.84,2243679.0,0.0,1.0,2022-03-01,1,3041.05,0.602414,1
2022-03-02,3016.98,3059.99,2975.00,3041.05,3041.05,2366736.0,0.0,1.0,2022-03-02,2,2957.97,-2.731951,0
2022-03-03,3070.63,3078.58,2937.12,2957.97,2957.97,3259860.0,0.0,1.0,2022-03-03,3,2912.82,-1.526385,0
2022-03-04,2943.18,2957.00,2876.14,2912.82,2912.82,3049156.0,0.0,1.0,2022-03-04,4,NaN,NaN,0


In [1052]:
df.increase.value_counts()

1    1623
0    1441
Name: increase, dtype: int64

In [1053]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [1054]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [1055]:
benzinga_csv_path = f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker}_finbert_20100101_20220304.csv"

In [1056]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [1057]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['Stock'])

In [1058]:
df_benzinga["day_of_week"] = df_benzinga["date"].dt.dayofweek

In [1059]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12394 entries, 0 to 12393
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     12394 non-null  object        
 1   Stock        12394 non-null  object        
 2   Positive     12394 non-null  float64       
 3   Negative     12394 non-null  float64       
 4   Neutral      12394 non-null  float64       
 5   date         12394 non-null  datetime64[ns]
 6   day_of_week  12394 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 677.9+ KB


In [1060]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Benzinga’s Volume Movers (FFBC, NFLX, GNTX, ALGN, UTHR) First Financial Bancorp shares gained 6.94% to $16.19. The volume of FFBC shares traded was 1311% higher than normal. FFBC has entered into an underwriting agreement for the sale of about 85 million common shares. Netflix Inc shares jumped 22.48% to $62.43. The volume of NFLX shares traded was 804% higher than normal. NFLX has hinted at expanding its operations to overseas markets. Gentex Corporation shares climbed 8.02% to $18.85. The volume of GNTX shares traded was 372% higher than normal. Robert W Baird has upgraded GNTX from “neutral” to “outperform.” Align Technology Inc shares rose 11.44% to $18.60. The volume of ALGN shares traded was 355% higher than normal. GNTX has posted a 45% increase in its net sales for Q4. United Therapeutics Corporation shares moved up 2.90% to $59.23. The volume of UTHR shares traded was 350% higher than normal. UTHR’s shares have surpassed their ten-day moving average of $58.23. Read more from Benzinga's Markets.",2010-01-28,0.952805,0.017886,0.029309,2010-01-28,3


In [1061]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12394 entries, 0 to 12393
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     12394 non-null  object        
 1   Stock        12394 non-null  object        
 2   Positive     12394 non-null  float64       
 3   Negative     12394 non-null  float64       
 4   Neutral      12394 non-null  float64       
 5   date         12394 non-null  datetime64[ns]
 6   day_of_week  12394 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 677.9+ KB


In [1062]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_benzinga.day_of_week.value_counts()

3    2712
2    2620
1    2358
0    2229
4    1980
6    283 
5    212 
Name: day_of_week, dtype: int64

In [1063]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Benzinga’s Volume Movers (FFBC, NFLX, GNTX, ALGN, UTHR) First Financial Bancorp shares gained 6.94% to $16.19. The volume of FFBC shares traded was 1311% higher than normal. FFBC has entered into an underwriting agreement for the sale of about 85 million common shares. Netflix Inc shares jumped 22.48% to $62.43. The volume of NFLX shares traded was 804% higher than normal. NFLX has hinted at expanding its operations to overseas markets. Gentex Corporation shares climbed 8.02% to $18.85. The volume of GNTX shares traded was 372% higher than normal. Robert W Baird has upgraded GNTX from “neutral” to “outperform.” Align Technology Inc shares rose 11.44% to $18.60. The volume of ALGN shares traded was 355% higher than normal. GNTX has posted a 45% increase in its net sales for Q4. United Therapeutics Corporation shares moved up 2.90% to $59.23. The volume of UTHR shares traded was 350% higher than normal. UTHR’s shares have surpassed their ten-day moving average of $58.23. Read more from Benzinga's Markets.",2010-01-28,0.952805,0.017886,0.029309,2010-01-28,3


In [1064]:
df_benzinga["in_index"] = df_benzinga["Stock"].apply(lambda x: True if x in df.index else False)

In [1065]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    283
5    212
0    31 
2    5  
4    4  
1    3  
Name: day_of_week, dtype: int64

In [1066]:
df.index.get_loc('2010-01-04')


0

In [1067]:
from datetime import date, datetime, timedelta

In [1068]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [1069]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Thursday
                days = timedelta(3)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Thursday
                days = timedelta(2)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Thursday
                days = timedelta(1)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Thursday
                days = timedelta(4)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc



In [1070]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_benzinga["backfill_date"] = df_benzinga["Stock"] # df_benzinga.apply(update_date, axis=1)


In [1071]:
ERROR_COUNTER

0

In [1072]:
df_benzinga["in_index"] = df_benzinga["backfill_date"].apply(lambda x: True if x in df.index else False)

In [1073]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    283
5    212
0    31 
2    5  
4    4  
1    3  
Name: day_of_week, dtype: int64

In [1074]:
df_benzinga[df_benzinga.in_index==False].shape

(538, 9)

In [1075]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [1076]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [1077]:
df_benzinga = df_benzinga[["backfill_date", "Positive", "Negative", "Neutral"]]

In [1078]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12394 entries, 0 to 12393
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  12394 non-null  object 
 1   Positive       12394 non-null  float64
 2   Negative       12394 non-null  float64
 3   Neutral        12394 non-null  float64
dtypes: float64(3), object(1)
memory usage: 387.4+ KB


In [1079]:
df_benzinga.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-28,0.952805,0.017886,0.029309
1,2010-03-10,0.773691,0.028808,0.197501
2,2010-03-29,0.096045,0.584013,0.319942
3,2010-04-01,0.066259,0.024507,0.909235
4,2010-05-17,0.028561,0.616754,0.354685


In [1080]:
df_benzinga.rename(columns={"backfill_date": "date"}, inplace=True)

In [1081]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [1082]:
df_benzinga.head()

,date,Positive,Negative,Neutral
0,2010-01-28,0.952805,0.017886,0.029309
1,2010-03-10,0.773691,0.028808,0.197501
2,2010-03-29,0.096045,0.584013,0.319942
3,2010-04-01,0.066259,0.024507,0.909235
4,2010-05-17,0.028561,0.616754,0.354685


In [1083]:
# df_benzinga = clean_tweet(df_benzinga)

In [1084]:
df_benzinga.set_index("date", inplace=True)

In [1085]:
df_benzinga.head()

,Positive,Negative,Neutral
date,,,
2010-01-28,0.952805,0.017886,0.029309
2010-03-10,0.773691,0.028808,0.197501
2010-03-29,0.096045,0.584013,0.319942
2010-04-01,0.066259,0.024507,0.909235
2010-05-17,0.028561,0.616754,0.354685


In [1086]:
df_benzinga_duplicated_index = df_benzinga[df_benzinga.index.duplicated(keep=False)]

In [1087]:
df_benzinga_duplicated_index.shape[0]

11770

In [1088]:
df_benzinga_nonduplicated_index = df_benzinga[~df_benzinga.index.duplicated(keep=False)]

In [1089]:
df_benzinga_nonduplicated_index.shape[0]

624

In [1090]:
# df_twint.groupby('date')['negative'].mean()

df_benzinga_groupby = df_benzinga.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [1091]:
df_benzinga_groupby

,negative,nuetral,positive
date,,,
2010-01-28,0.017886,0.029309,0.952805
2010-03-10,0.028808,0.197501,0.773691
2010-03-29,0.584013,0.319942,0.096045
2010-04-01,0.024507,0.909235,0.066259
2010-05-17,0.616754,0.354685,0.028561
...,...,...,...
2022-02-28,0.448063,0.392553,0.159384
2022-03-01,0.600509,0.265521,0.133970
2022-03-02,0.368508,0.540080,0.091412


In [1092]:
df_merge = pd.merge(df, df_benzinga_groupby, how="inner", left_index=True, right_index=True)

In [1093]:
df_merge.shape

(2400, 16)

In [1094]:
df_merge.day_of_week.value_counts()

1    495
2    494
3    482
4    470
0    459
Name: day_of_week, dtype: int64

In [1095]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [1096]:
df.shape

(3064, 13)

In [1097]:
df_benzinga_groupby.shape

(2782, 3)

In [1098]:
# df_merge.fillna(0.0, inplace=True)
# df_merge.fillna(method="ffill", inplace=True)

In [1099]:
df_merge[df_merge.negative.isnull()].shape

(0, 16)

In [1100]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2400 entries, 2010-01-28 to 2022-03-04
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               2400 non-null   float64       
 1   2. high               2400 non-null   float64       
 2   3. low                2400 non-null   float64       
 3   4. close              2400 non-null   float64       
 4   5. adjusted close     2400 non-null   float64       
 5   6. volume             2400 non-null   float64       
 6   7. dividend amount    2400 non-null   float64       
 7   8. split coefficient  2400 non-null   float64       
 8   date_time             2400 non-null   datetime64[ns]
 9   day_of_week           2400 non-null   int64         
 10  adjusted_close_shift  2399 non-null   float64       
 11  percentage_change     2399 non-null   float64       
 12  increase              2400 non-null   int64         
 13  negative

In [1101]:
# df_merge.fillna(0.0, inplace=True)

In [1102]:
##############df_merge.to_csv(f"/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/{ticker_upper}/{ticker_upper}_WITH_BENZINGA_FINBERT_SA.csv")
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-28,124.43,127.20,122.80,126.03,126.03,27293100.0,0.0,1.0,2010-01-28,3,125.410,-0.491946,0,0.017886,0.029309,0.952805
2010-03-10,129.11,131.17,128.48,130.51,130.51,5625700.0,0.0,1.0,2010-03-10,2,133.580,2.352310,1,0.028808,0.197501,0.773691
2010-03-29,135.37,136.63,134.33,135.12,135.12,4626900.0,0.0,1.0,2010-03-29,0,136.579,1.079781,1,0.584013,0.319942,0.096045
2010-04-01,135.80,136.51,131.18,131.81,131.81,8785800.0,0.0,1.0,2010-04-01,3,131.490,-0.242774,0,0.024507,0.909235,0.066259
2010-05-17,128.24,129.95,125.80,128.91,128.91,5642200.0,0.0,1.0,2010-05-17,0,126.280,-2.040183,0,0.616754,0.354685,0.028561


In [1103]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [1104]:
df_merge.shape

(2400, 16)

In [1105]:
# df_merge[(df_merge["negative"]==0.0) & (df_merge["nuetral"]==0.0) & (df_merge["positive"]==0.0)].shape

In [1106]:
# df_merge = df_merge[~(df_merge["negative"]==0.0) & ~(df_merge["nuetral"]==0.0) & ~(df_merge["positive"]==0.0)].copy()

In [1107]:
X = df_merge[["negative", "nuetral", "positive"]]
y = df_merge["increase"]

In [1108]:
y.head()

date
2010-01-28    0
2010-03-10    1
2010-03-29    1
2010-04-01    0
2010-05-17    0
Name: increase, dtype: int64

In [1109]:
from sklearn.model_selection import train_test_split

In [1110]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)



In [1111]:
SPLIT = int(0.8 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [1112]:
X_train

,negative,nuetral,positive
date,,,
2010-01-28,0.017886,0.029309,0.952805
2010-03-10,0.028808,0.197501,0.773691
2010-03-29,0.584013,0.319942,0.096045
2010-04-01,0.024507,0.909235,0.066259
2010-05-17,0.616754,0.354685,0.028561
...,...,...,...
2020-03-31,0.331296,0.302237,0.366467
2020-04-01,0.111791,0.863226,0.024983
2020-04-02,0.378922,0.504308,0.116769


In [1113]:
from sklearn.ensemble import RandomForestClassifier

In [1114]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [1115]:
preds = rfc.predict(X_test)

In [1116]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [1117]:
matrix = confusion_matrix(y_test, preds)

In [1118]:
matrix

array([[103, 124],
       [107, 146]])

In [1119]:
score = accuracy_score(y_test, preds)

In [1120]:
score

0.51875

In [1125]:
y_test.mean()

0.5270833333333333

In [1121]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.49      0.45      0.47       227
           1       0.54      0.58      0.56       253

    accuracy                           0.52       480
   macro avg       0.52      0.52      0.51       480
weighted avg       0.52      0.52      0.52       480



In [1122]:
corr_matrix = df_merge[["negative", "nuetral", "positive", "increase"]].corr()
print (corr_matrix)

          negative   nuetral  positive  increase
negative  1.000000 -0.639092 -0.438490 -0.003441
nuetral  -0.639092  1.000000 -0.411010  0.026577
positive -0.438490 -0.411010  1.000000 -0.026976
increase -0.003441  0.026577 -0.026976  1.000000


In [1123]:
from statsmodels.tsa.stattools import grangercausalitytests

In [1124]:
#perform Granger-Causality test
grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[1])


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=10.8093 , p=0.0010  , df_denom=2396, df_num=1
ssr based chi2 test:   chi2=10.8228 , p=0.0010  , df=1
likelihood ratio test: chi2=10.7985 , p=0.0010  , df=1
parameter F test:         F=10.8093 , p=0.0010  , df_denom=2396, df_num=1


{1: ({'ssr_ftest': (10.809256421552075, 0.0010245325738089497, 2396.0, 1),
   'ssr_chi2test': (10.822790548958023, 0.0010025826464876721, 1),
   'lrtest': (10.798450887380568, 0.0010158506649942394, 1),
   'params_ftest': (10.809256421551833, 0.0010245325738091333, 2396.0, 1.0)},
   array([[0., 1., 0.]])])}